In [1]:
#!/bin/python

import xml.etree.ElementTree as ET
import requests

# remove "{...}" stuff
def clear_tag(e):
    tag=e.tag
    return clear_tag_(tag)

# remove "{...}" stuff
def clear_tag_(tag):
    if "}" in tag: tag=tag.split("}")[1]
    return tag.lower()

# print for CSV
# todo: use some native CSV python methods
# todo: print to file instead to stdout
def print_formatted(text):
    if not isinstance(text,str):
        text=text.decode('utf-8')
    text=text.replace(","," ")
    text=text.replace("\n","*")
    text = text.replace("\r", "*")
    text=clear_tag_(text)
    # got some failures in shell for some reason, working around those
    # print (text.encode('utf-8')+",",end = '')
    print (text+",",end = '')

def newline():
    print ("")

# get the content of meeting
def parse_session_item(item):
    baseurl = 'http://knesset.gov.il/Odata/ParliamentInfo.svc/'
    url=baseurl+item
    r = requests.get(url)
    root = ET.fromstring(r.text.encode('utf-8'))
    text=""
    for e in root:
        if clear_tag(e)=='entry':
            for ee in e:
                if clear_tag(ee)=='content':
                    for eee in ee:
                        if clear_tag(eee)=='properties':
                            for eeee in eee:
                                if clear_tag(eeee)=='name':
                                    text+= eeee.text + " | "
    return text

def print_tag_text(entry,level=0):
    print("-"*level + entry.tag + "->" + entry.text)
# work on a feed.
# note - feed can contain follow up page (next_url).
def work_feed(feed, first=False, debug=False):
    # hold the next page
    next_url=None
    for cc in feed.getchildren():
        # next page is nested here
        if 'entry' not in cc.tag:
            if 'link' in cc.tag:
                if cc.attrib['rel'] == 'next':
                    assert(next_url is None) # just to be sure
                    next_url = cc.attrib['href']
            continue
        if debug: print_tag_text(cc,2)
        for ccc in cc.getchildren():
            if 'link' in ccc.tag:
                if 'KNS_CmtSessionItems' in ccc.attrib.get('title', ''):
                    # session description is found in this URL. Process it separately.
                    item = parse_session_item(ccc.attrib['href'])
            if debug: print_tag_text(ccc,3)
            # iterate over sessions.
            for cccc in ccc.getchildren():
                if debug:print_tag_text(cccc,4)
                if first and len(cccc.getchildren()):
                    # print field names. just once - first feed first session.
                    for ccccc in cccc.getchildren():
                        print_formatted(ccccc.tag)
                    first = False
                    print ("Description",end='')
                    newline()
                for ccccc in cccc.getchildren():
                    # all session items are interesting. print them all
                    if ccccc.text:
                        print_formatted(ccccc.text)
                    else:
                        print (",",end="")

                if len(cccc.getchildren()):
                    # print (last item - session description
                    assert(item is not None)
                    print_formatted(item.encode('utf_8'))
                    newline()

    return next_url

url='http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_Committee(926)?$expand=KNS_CommitteeSessions'
r = requests.get(url)
root = ET.fromstring(r.text.encode('utf-8'))
# this is ODATA root

debug = False # debug flag - print everything
first = True
for elem in root:
    if debug: print (">", elem.tag)

    if clear_tag(elem) == "content":
        # print information about document. this should be last printed lines.
        print ("*****")
        for c in elem.getchildren():
            for cc in c.getchildren():
                if cc.text: print_formatted(cc.tag), print_formatted(cc.text)

    # root
    # # elem
    # # # subelem
    # # # # c
    # # # # # cc - entry/link
    for subelem in elem:
        for c in subelem.getchildren():
            if debug: print ("-", c.tag, c.text)
            if 'feed' not in c.tag:
                continue

            # iterate over feeds
            while (c):
                next_feed=work_feed(c,first,debug)
                first=False
                if next_feed:
                    # get next feed
                    r = requests.get(next_feed)
                    c = ET.fromstring(r.text.encode('utf-8'))
                else:
                    c = None







/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:121: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/srv/conda/env

committeesessionid,number,knessetnum,typeid,typedesc,committeeid,location,sessionurl,broadcasturl,startdate,finishdate,note,lastupdateddate,Description
565346,1,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,http://main.knesset.gov.il/activity/committees/pages/allcommitteesagenda.aspx?tab=3&itemid=565346,http://main.knesset.gov.il/activity/committees/pages/allcommitteesbroadcast.aspx?topicid=8444,2015-06-03t12:00:00,,,2019-04-28t19:40:50.3766667,בחירת יושב ראש הוועדה  בהתאם לסעיף 106(א)(1) לתקנון הכנסת | ,
565375,2,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,http://main.knesset.gov.il/activity/committees/pages/allcommitteesagenda.aspx?tab=3&itemid=565375,http://main.knesset.gov.il/activity/committees/pages/allcommitteesbroadcast.aspx?topicid=8464,2015-06-08t11:00:00,,הישיבה תשודר בערוץ הכנסת,2019-04-28t19:40:50.3766667,המצב ברשות השידור ויישום חוק השידור הציבורי | ,
565380,4,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,ht

KeyboardInterrupt: 

In [3]:
from extract_py3 import process_url
url='http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_Committee(926)?$expand=KNS_CommitteeSessions'
# url='http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_Committee(926)/KNS_CommitteeSessions'
url=process_url(url)


committeesessionid,number,knessetnum,typeid,typedesc,committeeid,location,sessionurl,broadcasturl,startdate,finishdate,note,lastupdateddate,Description
565346,1,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,http://main.knesset.gov.il/activity/committees/pages/allcommitteesagenda.aspx?tab=3&itemid=565346,http://main.knesset.gov.il/activity/committees/pages/allcommitteesbroadcast.aspx?topicid=8444,2015-06-03t12:00:00,,,2019-04-28t19:40:50.3766667,בחירת יושב ראש הוועדה  בהתאם לסעיף 106(א)(1) לתקנון הכנסת | ,
565375,2,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,http://main.knesset.gov.il/activity/committees/pages/allcommitteesagenda.aspx?tab=3&itemid=565375,http://main.knesset.gov.il/activity/committees/pages/allcommitteesbroadcast.aspx?topicid=8464,2015-06-08t11:00:00,,הישיבה תשודר בערוץ הכנסת,2019-04-28t19:40:50.3766667,המצב ברשות השידור ויישום חוק השידור הציבורי | ,
565380,4,20,161,פתוחה,926,חדר הוועדה  באגף הוועדות (קדמה)  קומה 3  חדר 3740,ht

KeyboardInterrupt: 